In [5]:
import os
import glob
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
from sublimpy import utils, extrautils
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
import act

## Horizontal wind - profiles

In [6]:
USERNAME = os.getenv("ARM_USERNAME")
TOKEN = os.getenv("ARM_TOKEN")
SAIL_DATA_STREAM = 'gucdlprofwind4newsM1.c1'
DATA_STREAM_FILEEXT = '.nc'
startdate = "2023-02-01"
enddate = "2023-06-01"
output_dir = os.path.join("/Users/elischwat/Development/data/sublimationofsnow/", SAIL_DATA_STREAM)


In [7]:
act.discovery.download_arm_data(USERNAME, TOKEN, SAIL_DATA_STREAM, startdate, enddate, output=output_dir)

[DOWNLOADING] gucdlprofwind4newsM1.c1.20230404.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230412.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230511.001420.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230331.001420.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230506.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230405.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230309.001420.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230213.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230526.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230303.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230202.000039.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230301.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230414.001420.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230214.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230222.001421.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230421.001420.nc
[DOWNLOADING] gucdlprofwind4newsM1.c1.20230224.001421.nc
[DOWNLOADING] gucdlprofwind4new

['/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230404.001421.nc',
 '/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230412.001421.nc',
 '/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230511.001420.nc',
 '/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230331.001420.nc',
 '/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230506.001421.nc',
 '/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230405.001421.nc',
 '/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230309.001420.nc',
 '/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20230213.001421.nc',
 '/Users/elischw

In [8]:
dl_w_prof_files = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/gucdlprofwind4newsM1.c1/*.nc")

In [9]:
prof_ds = act.io.read_arm_netcdf(dl_w_prof_files)
prof_ds.time.min(), prof_ds.time.max()

(<xarray.DataArray 'time' ()> Size: 8B
 array('2023-02-01T00:00:39.881244000', dtype='datetime64[ns]'),
 <xarray.DataArray 'time' ()> Size: 8B
 array('2023-06-15T23:59:20.789108000', dtype='datetime64[ns]'))

In [10]:
prof_df = prof_ds.to_dataframe()
prof_df = utils.modify_df_timezone(prof_df.reset_index(), 'UTC', 'US/Mountain').set_index('time')

In [11]:
hourly_mean_prof_df = prof_df.query("height > 950").query("height < 1100")[['wind_speed', 'wind_direction']].reset_index().set_index('time').groupby(pd.Grouper(freq='60min')).mean()

In [12]:
synoptic_winds_30min = hourly_mean_prof_df.resample('30min').mean().ffill()
along_valley_wind_times = synoptic_winds_30min[
    (synoptic_winds_30min.wind_direction > 299)
    & (synoptic_winds_30min.wind_direction < 345 )
].index.values
cross_valley_wind_times = synoptic_winds_30min[
    (synoptic_winds_30min.wind_direction > 202)
    & (synoptic_winds_30min.wind_direction < 248 )
].index.values
len(cross_valley_wind_times), len(along_valley_wind_times)

(1648, 1040)

# Surface Measurements

In [13]:
sos_tidy_fn = f"../paper1/process_slow_data/tidy_df_20221101_20230619_planar_fit_multiplane_STRAIGHTUP_q7_flags9000_pf10.parquet"
tidy_df = pd.read_parquet(sos_tidy_fn)
# Convert data timezone to local and clean up data on the ends
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')
tidy_df = tidy_df[tidy_df.time > '20221107']

In [14]:
src = tidy_df[tidy_df.variable.isin(['tke_3m_c', 'spd_3m_c', 'w_h2o__3m_c', 'w_tc__3m_c'])].pivot(index='time', columns='variable', values='value')
src = pd.concat([
        src[src.index.isin(cross_valley_wind_times)].assign(synoptic_wind = 'cross valley'),
        src[src.index.isin(along_valley_wind_times)].assign(synoptic_wind = 'along valley')
    ])
src['sqrt(tke)'] = src['tke_3m_c']**0.5
turbulence_regime_scatter_chart = alt.Chart(
    src
).mark_circle(size=5, color='orange').encode(
    alt.X('spd_3m_c').scale(domain=[0, 10], clamp=True),
    alt.Y('sqrt(tke)'),
    alt.Color('synoptic_wind:N')
).properties(width=200, height = 200)
turbulence_regime_scatter_chart

alt.Chart(...)

In [15]:
src['spd bin'] = pd.cut(
    src['spd_3m_c'], 
    np.arange(0,11), 
    # labels = [f"{lo}-{hi}" for lo, hi in zip(np.arange(0,11), np.arange(1,11))]
    labels = [(lo + hi)/2 for lo, hi in zip(np.arange(0,11), np.arange(1,11))]
)


In [16]:
turbulence_regime_box_chart = alt.Chart(
    src
).mark_boxplot(outliers=False, opacity=0.5, box={'stroke': 'black'}).encode(
    alt.X('spd bin:Q').title('3m wind speed (m/s)'),
    alt.Y('sqrt(tke):Q'),
    alt.Color('synoptic_wind:N'),
).properties(width=200, height = 200)

In [17]:
(
    windspeed_casestudy_chart 
    | (
        regime_casestudy_chart 
        | (turbulence_regime_scatter_chart + turbulence_regime_box_chart)
    ).resolve_scale(y='shared', x='shared', color='independent')
).resolve_scale(color='independent')

NameError: name 'windspeed_casestudy_chart' is not defined

In [ ]:
(alt.Chart(
    src
).mark_boxplot(outliers=False, opacity=0.5, box={'stroke': 'black'}).encode(
    alt.X('spd bin:Q').title('3m wind speed (m/s)'),
    alt.Y('w_h2o__3m_c:Q').title('TKE at 3m (m²/s²)'),
    alt.Color('synoptic_wind:N'),
).properties(width=200, height = 200) | alt.Chart(
    src
).mark_boxplot(outliers=False, opacity=0.5, box={'stroke': 'black'}).encode(
    alt.X('spd bin:Q').title('3m wind speed (m/s)'),
    alt.Y('w_h2o__20m_c:Q').title('TKE at 3m (m²/s²)'),
    alt.Color('synoptic_wind:N'),
).properties(width=200, height = 200)).resolve_scale(x='shared', y='shared')

In [ ]:
(alt.Chart(
    src
).mark_boxplot(outliers=False, opacity=0.5, box={'stroke': 'black'}).encode(
    alt.X('spd bin:Q').title('3m wind speed (m/s)'),
    alt.Y('w_tc__3m_c:Q').title('TKE at 3m (m²/s²)'),
    alt.Color('synoptic_wind:N'),
).properties(width=200, height = 200) | alt.Chart(
    src
).mark_boxplot(outliers=False, opacity=0.5, box={'stroke': 'black'}).encode(
    alt.X('spd bin:Q').title('3m wind speed (m/s)'),
    alt.Y('w_tc__20m_c:Q').title('TKE at 3m (m²/s²)'),
    alt.Color('synoptic_wind:N'),
).properties(width=200, height = 200)).resolve_scale(x='shared', y='shared')

In [ ]:
turbulence_regime_box_chart

In [ ]:
src['sqrt(tke)'] = src['tke_3m_c']**(0.5)
turbulence_regime_box_chart = alt.Chart(
    src
).mark_boxplot(opacity=0.5, box={'stroke': 'black'}, outliers=False).encode(
    alt.X('spd bin:Q').title('3m wind speed (m/s)'),
    alt.Y('sqrt(tke):Q').title('TKE at 3m (m²/s²)'),
    alt.Color('synoptic_wind:N'),
).properties(width=200, height = 200)

turbulence_regime_chart = (turbulence_regime_scatter_chart + turbulence_regime_box_chart).resolve_scale(
    x='shared'
)
turbulence_regime_chart

In [ ]:
synoptic_winds_pdf_chart =alt.Chart(
    pd.concat([
        synoptic_winds_30min[synoptic_winds_30min.index.isin(cross_valley_wind_times)].assign(synoptic_wind = 'cross valley'),
        synoptic_winds_30min[synoptic_winds_30min.index.isin(along_valley_wind_times)].assign(synoptic_wind = 'along valley')
    ])
).transform_density(
    'wind_speed',
    as_ = ['wind speed', 'density'],
    groupby=['synoptic_wind']
).mark_line().encode(
    alt.X('wind speed:Q').title('1000m wind speed (m/s)'),
    alt.Y('density:Q').title('PDF (kernel density estimate)'),
    alt.Color('synoptic_wind:N').title(['Synoptic wind', 'direction'])
).properties(width = 200, height = 200)
synoptic_winds_pdf_chart

In [ ]:
turbulence_regime_chart | synoptic_winds_pdf_chart

In [ ]:
src = pd.concat([
    tidy_df.query("measurement == 'temperature'"),
    tidy_df.query("variable == 'Tsurf_c'"),
])

src = src[src.time <= '20230411 1600'][src.time >= '20230411 0900']
alt.Chart(src).mark_circle().encode(
    alt.X('value:Q').title('temp. (˚C)').axis(values=[0,2,4,6,8,10]),
    alt.Y('height:Q').title('height (m)'),
    alt.Facet('time:T', columns=5).header(format='April %d %H:%M')
).properties(width=100, height=100).display(renderer='svg')

In [ ]:
src = pd.concat([
    tidy_df.query("measurement == 'temperature'"),
    tidy_df.query("variable == 'Tsurf_c'"),
])

src = src[src.time <= '20230211 1600'][src.time >= '20230211 0900']
alt.Chart(src).mark_circle().encode(
    alt.X('value:Q').title('temp. (˚C)'),
    alt.Y('height:Q').title('height (m)'),
    alt.Facet('time:T', columns=5).header(format='February %d %H:%M')
).properties(width=100, height=100).display(renderer='svg')

# Local Scaling - does MOST work ?

Calculate dimensionless wind gradient (stability correction) and the stability function (z/L)

In [ ]:
# src = tidy_df [tidy_df.time > '20230415'][tidy_df.time < '20230418']
HEIGHT = 3
src = tidy_df.copy()
src = src[src.variable.isin([
    'u*_3m_c', 'spd_3m_c', 'L_3m_c', 'Tvirtual_3m_c', 'w_tc__3m_c', 'wind_gradient_3m_c', 'temp_gradient_3m_c'
])]
src = src.pivot_table(values='value', columns='variable', index='time')
src['L_3m_c'] = - (src['u*_3m_c']**3)*(src['Tvirtual_3m_c']+273.5) / (0.4 * 9.81 * src['w_tc__3m_c'])
src['z_over_l'] = HEIGHT / src['L_3m_c']
src['stability_correction'] = 0.4*HEIGHT*src['wind_gradient_3m_c']/src['u*_3m_c']

src_along_valley_times = src[src.index.isin(along_valley_wind_times)]
src_cross_valley_times = src[src.index.isin(cross_valley_wind_times)]

Create a fit line for a couple different stability corrections

In [ ]:
def holtslag_and_debruin(zeta):
    return (
        1 + 0.7*zeta + 0.75*zeta*(6 - 0.35*zeta)*np.exp(-0.35*zeta)
    )
def lettau(zeta):
    return (1 + 4.5*zeta)**0.75
fits_domain_stable = pd.Series(np.linspace(0,2))
fits_domain_unstable = pd.Series(np.linspace(-2,0))
fits_df = pd.concat([
    pd.DataFrame({
        'z_over_l': fits_domain_stable,
        'stability_correction': fits_domain_stable.apply(lettau),
    }),
    pd.DataFrame({
        'z_over_l': fits_domain_unstable,
        'stability_correction': (1 - 15 * fits_domain_unstable)**(-0.25)
    })
])

In [ ]:
alt.Chart(
    src
).mark_circle(size=2).encode(
    alt.X('z_over_l').scale(domain=[-2, 2], clamp=True),
    alt.Y('stability_correction').scale(domain=[0, 6], clamp=True)
).properties(title='All data') +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
).properties(width=250, height = 250)

alt.LayerChart(...)

In [ ]:
y_max = 3.5
x_min = -1.0
x_max = 1.0
scatter_chart_base = alt.Chart(
).mark_circle(size=10).encode(
    alt.X('z_over_l').scale(domain=[x_min, x_max], clamp=True),
    alt.Y('stability_correction').scale(domain=[0, y_max], clamp=True)
).properties(title='All data') +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q').scale(domain=[x_min, x_max], clamp=True),
    alt.Y('stability_correction:Q').scale(domain=[0, y_max], clamp=True)
).properties(width=250, height = 250)

alt.layer(scatter_chart_base, 
    data=src_along_valley_times.query(f"z_over_l < {x_max}").query(f"z_over_l > {x_min}").query("stability_correction > 0").query(f"stability_correction < {y_max}")
).properties(title='Along valley winds') | alt.layer(scatter_chart_base, 
    data=src_cross_valley_times.query(f"z_over_l < {x_max}").query(f"z_over_l > {x_min}").query("stability_correction > 0").query(f"stability_correction < {y_max}")
).properties(title='Cross valley winds')

alt.HConcatChart(...)

In [ ]:
import altair as alt

# Define the base chart for 2D histograms
histogram_base = alt.Chart().mark_rect().encode(
    alt.X('z_over_l:Q', bin=alt.Bin(step=0.1), scale=alt.Scale(domain=[-1, 1], clamp=True)),
    alt.Y('stability_correction:Q', bin=alt.Bin(step=0.1), scale=alt.Scale(domain=[0, 3], clamp=True)),
    alt.Color('count():O')
).properties(width=250, height=250)

# Create the line chart for fits_df
line_chart = alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
)

# Combine the histograms and line chart
chart_along_valley = alt.layer(histogram_base, line_chart, data=src_along_valley_times).properties(title='Along valley winds')
chart_cross_valley = alt.layer(histogram_base, line_chart, data=src_cross_valley_times).properties(title='Cross valley winds')

# Display the charts side by side
chart_along_valley | chart_cross_valley

alt.HConcatChart(...)

In [ ]:
all_data_chart = (alt.Chart(
    src.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10")
).mark_boxplot(outliers=False).encode(
    alt.X('z_over_l:Q').bin(step=0.2),
    alt.Y('stability_correction:Q')
) +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
)).properties(title='All Data', width=250, height = 250)

along_valley_chart = (alt.Chart(
    src_along_valley_times.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10")
).mark_boxplot(outliers=False).encode(
    alt.X('z_over_l:Q').bin(step=0.2),
    alt.Y('stability_correction:Q')
) +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
)).properties(title='Along-valley synoptic winds', width=250, height = 250)
cross_valley_chart = (alt.Chart(
    src_cross_valley_times.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10")
).mark_boxplot(outliers=False).encode(
    alt.X('z_over_l:Q').bin(step=0.2),
    alt.Y('stability_correction:Q')
) +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
)).properties(title='Cross-valley synoptic winds', width=250, height = 250)
all_data_chart | along_valley_chart | cross_valley_chart

alt.HConcatChart(...)

In [ ]:
all_data_chart = (alt.Chart(
    src.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10")
).mark_circle().encode(
    alt.X('z_over_l:Q').bin(step=0.2),
    alt.Y('median(stability_correction):Q')
) +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
)).properties(title='All Data', width=250, height = 250)

along_valley_chart = (alt.Chart(
    src_along_valley_times.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10")
).mark_point().encode(
    alt.X('z_over_l:Q').bin(step=0.2),
    alt.Y('median(stability_correction):Q')
) +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
)).properties(title='Along-valley synoptic winds', width=250, height = 250)
cross_valley_chart = (alt.Chart(
    src_cross_valley_times.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10")
).mark_point(shape='square').encode(
    alt.X('z_over_l:Q').bin(step=0.2),
    alt.Y('median(stability_correction):Q')
) +\
    alt.Chart(fits_df).mark_line(color='red').encode(
    alt.X('z_over_l:Q'),
    alt.Y('stability_correction:Q')
)).properties(title='Cross-valley synoptic winds', width=250, height = 250)
all_data_chart | along_valley_chart | cross_valley_chart

alt.HConcatChart(...)

In [ ]:
combo_df = pd.concat([
    # src.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10").assign(category = 'all data'),
    src_along_valley_times.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10").assign(category = 'along valley'),
    src_cross_valley_times.query("z_over_l < 2").query("z_over_l > -2").query("stability_correction < 10").assign(category = 'cross valley'),
])

(
    # case study plot
    ####################################
    alt.Chart(
        pd.concat([
            src.loc['20230415'],
            src.loc['20230418']
        ]).reset_index().query("z_over_l < 2")
    ).mark_point(opacity=.5).encode(
        alt.X('z_over_l').scale(domain=[-2, 2], clamp=True),
        alt.Y('stability_correction').scale(domain=[0, 6], clamp=True),
        alt.Color('date(time):N')
    ) +\
        alt.Chart(fits_df).mark_line(color='red').encode(
        alt.X('z_over_l:Q'),
        alt.Y('stability_correction:Q')
    ).properties(title='Two days in April', width=150, height = 250)
    
    # summary study plot
    ####################################
    |
    (alt.Chart(
        combo_df
    ).mark_point().encode(
        alt.X('z_over_l:Q').bin(step=0.2).axis(values=[-2,-1,0,1,2], format='.0f', tickCount=5),
        alt.Y('median(stability_correction):Q'),
        alt.Color('category:N'),
    ) + alt.Chart(fits_df).mark_line(color='red').encode(
        alt.X('z_over_l:Q'),
        alt.Y('stability_correction:Q')
    )).properties(title='All Data', width=150, height = 250) 

).resolve_scale(color='independent').display(renderer='svg')